In [ ]:
import numpy as np
import cv2
from scipy.ndimage import sobel

In [ ]:
def diff_squared(strip1, strip2):
    """
    Computes the squared difference between two 2D RGB strips.
    Both strips must be the same shape: (N, 3).

    Returns:
        The sum of squared differences (SSD) as a number.
    """
    strip1 = np.asarray(strip1, dtype=np.float32)
    strip2 = np.asarray(strip2, dtype=np.float32)

    # Calculate (A - B)^2 for each RGB component and sum
    return np.sum((strip1 - strip2) ** 2)

In [ ]:
def gradient_error(strip1, strip2):
    """
    Computes an error value indicating how well two image strips connect based on gradients.

    Parameters:
        strip1: np.ndarray
            The first image strip (H x W x C) or (H x W) for grayscale.
        strip2: np.ndarray
            The second image strip (H x W x C) or (H x W) for grayscale.
            Assumes strip1's right edge should connect to strip2's left edge.

    Returns:
        float: Mean squared error of the gradients at the connecting edge.
    """

    # If color image, convert to grayscale
    if strip1.ndim == 3:
        strip1_gray = cv2.cvtColor(strip1, cv2.COLOR_BGR2GRAY)
        strip2_gray = cv2.cvtColor(strip2, cv2.COLOR_BGR2GRAY)
    else:
        strip1_gray = strip1
        strip2_gray = strip2

    # Compute gradients along the edge
    grad1 = sobel(strip1_gray)
    grad2 = sobel(strip2_gray)

    # Compute mean squared error between gradients
    error = np.mean((grad1 - grad2)**2)

    return error

In [ ]:
def z_score(strip):
    """
    Computes the Z-score for a strip.
    strip: numpy array of shape (H, W, 3) or (H, W)
    Returns a flattened 1D Z-scored array (for comparison)
    """

    if len(strip.shape) == 3:
        strip_gray = cv2.cvtColor(strip, cv2.COLOR_BGR2GRAY)
    else:
        strip_gray = strip.copy()


    strip_flat = strip_gray.flatten()


    mean = np.mean(strip_flat)
    std = np.std(strip_flat)
    if std == 0:
        std = 1e-6

    z = (strip_flat - mean) / std
    return z

In [ ]:
def compute_corr_error(stripA, stripB):
    zA = z_score(stripA)
    zB = z_score(stripB)

    min_len = min(len(zA), len(zB))
    zA = zA[:min_len]
    zB = zB[:min_len]

    diff = zA - zB
    Euclidean_error = np.sqrt(np.sum(diff**2))
    mae_error = np.mean(np.abs(diff))
    correlation = np.sum(zA * zB) / min_len

    return Euclidean_error, mae_error, correlation